In [2]:
import sys
sys.path.append('../')

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle as pkl

import util_gcnn
import util_results

plt.rcParams.update({"font.size":16})

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
project_dir = "/home/jtl/Dropbox (MIT)/project_uncertainty_quantification/"
out_folder = "2104_rc_new1/"
result_file = "rail_catchment_mve_results_val.csv"

## Benchmark

In [4]:
res_before, res_after = util_results.get_bmk_results(project_dir, out_folder, 
                        metrics=['nll_loss','mae','rmse','u','um','us','uc','picp','mpiw'], filt={'lookback':2})

## GCN

In [5]:
results = pd.read_csv(project_dir+"results/"+out_folder+result_file)
results = results[results['test_nll_loss']<1000]
results['val_rmse'] = np.sqrt(results['val_mse'])
results['test_rmse'] = np.sqrt(results['test_mse'])
results['Adjacency'] = pd.Categorical(results['Adjacency'], ['func','euc','con','net','func_euc','func_con','func_net','func_euc_con_net'])

before = results[(results['Period']=='before')&(results['Adjacency']=='func_euc_con_net')&(results['Lookback']==2)]
before = before.sort_values(by='val_nll_loss').iloc[0]

after = results[(results['Period']=='after')&(results['Adjacency']=='func_euc_con_net')&(results['Lookback']==2)]
after = after.sort_values(by='val_nll_loss').iloc[0]

for me in ['nll_loss','mae','rmse','u','um','us','uc','picp','mpiw']:
    res_before['gcn_'+me] = before['test_'+me]
    res_after['gcn_'+me] = after['test_'+me]

In [6]:
len(results)

955

## GCN Ensemble

In [7]:
with open(project_dir+'results/'+out_folder+'before_dt_gcn_ensemble.pkl', 'rb') as f:
    res_before.update(pkl.load(f))

In [8]:
with open(project_dir+'results/'+out_folder+'after_dt_gcn_ensemble.pkl', 'rb') as f:
    res_after.update(pkl.load(f))

## GCN-Homo

In [9]:
out_folder = "2106_rc_homo/"
result_file = "rail_catchment_mve_results.csv"

results = pd.read_csv(project_dir+"results/"+out_folder+result_file)
results = results[results['test_nll_loss']<1000]
results['val_rmse'] = np.sqrt(results['val_mse'])
results['test_rmse'] = np.sqrt(results['test_mse'])
results['Adjacency'] = pd.Categorical(results['Adjacency'], ['func','euc','con','net','func_euc','func_con','func_net','func_euc_con_net'])

before = results[(results['Period']=='before')&(results['Adjacency']=='func_euc_con_net')&(results['Lookback']==2)]
before = before.sort_values(by='val_nll_loss').iloc[0]

after = results[(results['Period']=='after')&(results['Adjacency']=='func_euc_con_net')&(results['Lookback']==2)]
after = after.sort_values(by='val_nll_loss').iloc[0]

for me in ['nll_loss','mae','rmse','u','um','us','uc','picp','mpiw']:
    res_before['gcn_homo_'+me] = before['test_'+me]
    res_after['gcn_homo_'+me] = after['test_'+me]

In [10]:
out_folder = "2106_rc_homo1/"
result_file = "rail_catchment_mve_results.csv"

results = pd.read_csv(project_dir+"results/"+out_folder+result_file)
#results = results[results['test_nll_loss']<1000]
results['val_rmse'] = np.sqrt(results['val_mse'])
results['test_rmse'] = np.sqrt(results['test_mse'])
results['Adjacency'] = pd.Categorical(results['Adjacency'], ['func','euc','con','net','func_euc','func_con','func_net','func_euc_con_net'])

before = results[(results['Period']=='before')&(results['Adjacency']=='func_euc_con_net')&(results['Lookback']==2)]
before = before.sort_values(by='val_nll_loss').iloc[0]

after = results[(results['Period']=='after')&(results['Adjacency']=='func_euc_con_net')&(results['Lookback']==2)]
after = after.sort_values(by='val_nll_loss').iloc[0]

for me in ['nll_loss','mae','rmse','u','um','us','uc','picp','mpiw']:
    res_before['gcn_homo1_'+me] = before['test_'+me]
    res_after['gcn_homo1_'+me] = after['test_'+me]

# Table: Model Performance

In [11]:
# hardcoded during revision
before_test_avg = 635.4
after_test_avg = 89.0

In [25]:
metric_format = {'mae':'{0:.4}','mape':'{0:.3}', 'rmse':'{0:.4}','u':'{0:.4}','um':'{0:.4}','us':'{0:.4}','uc':'{0:.4}',
                 'nll_loss':'{0:.4}','picp':'{0:.4}','mpiw':'{0:.4}'}
model_name = {'lw': 'Last Week', 'wls': 'Weighted Least Squares', 'gcn_homo':'GCN MVE Homo EV', 'gcn_homo1':'GCN MVE Homo 1',
              'gcn':'GCN MVE', 'gcn_ens':'GCN MVE Ensemble'}

for mo in ['lw', 'wls', 'gcn_homo', 'gcn_homo1', 'gcn', 'gcn_ens']:
    print(model_name[mo], end=' & ')
    for me in ['nll_loss','mpiw','picp','mae','mape']:
        end = ' & '
        if me == 'mape':
            print(metric_format[me].format(res_before[mo+'_'+'mae'] / before_test_avg * 100) + "\%", end='\\\\\n')
            
        elif mo+'_'+me in res_before.keys():
            print(metric_format[me].format(res_before[mo+'_'+me]), end=end)
        else:
            print('', end=end)

print('\midrule')
print('\multicolumn{8}{c}{During COVID (April 2020 - Oct 2020)}\\\\')
print('\midrule')

for mo in ['lw', 'wls', 'gcn_homo', 'gcn_homo1', 'gcn', 'gcn_ens']:
    print(model_name[mo], end=' & ')
    for me in ['nll_loss','mpiw','picp','mae','mape']:
        end = ' & '
        if me == 'mape':
            print(metric_format[me].format(res_after[mo+'_'+'mae'] / after_test_avg * 100) + "\%", end='\\\\\n')
        elif mo+'_'+me in res_after.keys():
#             if me == 'uc':
#                 end='\\\\\n'
#             if me in ['um','us']:
#                 end = ' / '
            print(metric_format[me].format(res_after[mo+'_'+me]), end=end)
        else:
            print('', end=end)

Last Week & 100.4 & 517.5 & 0.8994 & 71.82 & 11.3\%\\
Weighted Least Squares & 142.2 & 238.9 & 0.7356 & 77.94 & 12.3\%\\
GCN MVE Homo EV & 122.7 & 1.138e+03 & 0.9883 & 157.3 & 24.7\%\\
GCN MVE Homo 1 & 2.862e+04 & 14.37 & 0.206 & 63.07 & 9.93\%\\
GCN MVE & 94.38 & 356.2 & 0.9524 & 67.57 & 10.6\%\\
GCN MVE Ensemble & 94.1 & 397.1 & 0.9622 & 66.79 & 10.5\%\\
\midrule
\multicolumn{8}{c}{During COVID (April 2020 - Oct 2020)}\\
\midrule
Last Week & 75.09 & 109.3 & 0.9241 & 13.35 & 15.0\%\\
Weighted Least Squares & 115.7 & 48.6 & 0.5943 & 13.77 & 15.5\%\\
GCN MVE Homo EV & 80.22 & 85.81 & 0.9654 & 19.08 & 21.4\%\\
GCN MVE Homo 1 & 325.7 & 18.2 & 0.606 & 13.9 & 15.6\%\\
GCN MVE & 70.7 & 84.7 & 0.9559 & 13.97 & 15.7\%\\
GCN MVE Ensemble & 70.44 & 80.89 & 0.9616 & 13.46 & 15.1\%\\
